## 安裝套件

In [1]:
!pip install jieba
!pip install emoji langdetect
!pip install datasets
!pip install lingua-language-detector

In [2]:
import sys
!{sys.executable} -m pip install jieba emoji langdetect pytz torch lingua-language-detector datasets openpyxl


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


## 引入套件

In [8]:
pip install transformers

  Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 17.2 MB/s eta 0:00:00 0:00:01
Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl (284 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

import re
import emoji
import langdetect
from langdetect import detect
from lingua import LanguageDetectorBuilder, Language, IsoCode639_1
from datetime import datetime
import pytz
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
import torch

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 設定好路徑 (後面都是使用相對路徑)
base_path = '/content/drive/My Drive/SMA'

Mounted at /content/drive


In [2]:
# 讀取資料（請確認你的 Excel 路徑）
# df = pd.read_excel(base_path+"/threads.xlsx")
df = pd.read_excel("threads.xlsx", engine='openpyxl')
df

,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,share_count,view_count,followers_count,post_url,scrape_time
0,ayofvr,2025-04-29T10:58:39.000Z,NaN,3小時,Thank you God for another day.,N,N,190,3,23,NaN,"3,197",141073,https://www.threads.net/@ayofvr/post/DJBymf8uTrK,2025-04-29T22:27:40.176749
1,NaN,2025-04-29T12:33:44.000Z,NaN,1小時,百達翡麗？ 沒有下限的網路病態！,Y,N,196,16,NaN,6,3 萬,77683,https://www.threads.net/@ban.mei.onnnnni/post/...,2025-04-29T22:27:54.964788
2,ribboworld2021,2025-04-29T04:39:46.000Z,小一日常,9小時,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,N,N,75,6,NaN,NaN,"4,559",99,https://www.threads.net/@ribboworld2021/post/D...,2025-04-29T22:28:09.873641
3,ayofvr,2025-04-29T11:25:22.000Z,NaN,3小時,Just be strong. Confident. Hopeful. Intellectu...,N,N,83,5,3,1,"1,967",141093,https://www.threads.net/@ayofvr/post/DJB1qP5OmzP,2025-04-29T22:28:24.726576
4,jose_ykc,2025-04-29T10:36:19.000Z,輔仁大學,3小時,日文輔系老師上課內容之一AiScReam 歌詞導讀,Y,Y,"4,334",55,513,"2,460",10 萬,65,https://www.threads.net/@jose_ykc/post/DJBvpGI...,2025-04-29T22:28:39.393706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,2025年04月29日 07:30,NaN,8小時,First tasting in California,N,N,7,NaN,NaN,0,726,30,https://www.threads.net/@leighton.williams/pos...,2025-04-29T16:23:40.328046
3097,cape__man,2025年04月29日 06:31,NaN,9小時,I hoped it would have been better.,N,N,2,NaN,NaN,0,"1,291",114,https://www.threads.net/@cape__man/post/DJAdFd...,2025-04-29T16:23:55.063517
3098,simimoonlight,2025年04月29日 06:26,NaN,9小時,I can’t wait to watch Beyoncé on TikTok tonigh...,N,N,"2,656",29,280,47,5.6 萬,65732,https://www.threads.net/@simimoonlight/post/DJ...,2025-04-29T16:24:39.870994
3099,other98,2025年04月29日 12:31,NaN,3小時,"Tonight, Canada just proved that they have a h...",N,N,1.0 萬,214,214,22,14 萬,217182,https://www.threads.net/@other98/post/DJBGV3NxiX_,2025-04-29T16:24:54.669936


In [3]:
# === 語言偵測修正版===
lingua_detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()
lingua_available = True
def detect_lang_with_preprocessing_lingua(text):
    original_text = text

    # 若是 NaN 或空字串就回傳 "unknown"
    if pd.isna(text):
        return "unknown"
    text = str(text).strip()
    if not text:
        return "unknown"
    
    # 移除 URL、@標記、 #hashtag、emoji、多餘空白
    try:
      text_cleaned = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
      text_cleaned = re.sub(r'@\w+', '', text_cleaned)
      text_cleaned = re.sub(r'#\w+', '', text_cleaned)
      text_cleaned = emoji.replace_emoji(text_cleaned, replace='')
      text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()
    except Exception as e:
      return "error_state_preprocessing"

    # 若這些清理完後變成空字串
    if not text_cleaned:
      return "empty_after_clean"

    # 若文字中超過 30% 是中文，就直接判定為 "Ch"（中文）
    try:
      chinese_chars = re.findall(r'[\u4e00-\u9fff]', text_cleaned)
      text_len = len(text_cleaned)
      ratio = len(chinese_chars) / max(text_len, 1)
      chinese_threshold = 0.3
      if ratio > chinese_threshold:
        return "Ch"

      # 呼叫 lingua 偵測語言
      detected_language = lingua_detector.detect_language_of(text_cleaned)

      # 若 lingua 判定是中文（'ZH'），則回傳 "Ch"，其餘語言以小寫的 ISO 639-1 回傳（如 en, ja, fr）
      # 若無法偵測出語言，回傳 "unknown"
      if detected_language is not None:
        iso_code = detected_language.iso_code_639_1.name
        if iso_code == 'ZH':
          return "Ch"
        else:
          return iso_code.lower()
      else:
        return "unknown"

    except Exception as e:
      return "unknown"

## 清洗數據V1

In [4]:
# === 數值欄位清洗（萬字、逗號格式處理）===
def parse_count(value):
    # 將文字數字（如 "1,234"、"2.5萬"）統一轉為整數（int）
    if pd.isna(value): return 0
    value = str(value).replace(",", "")
    # "萬" 的部分會乘上 10,000 做轉換
    # 無法處理的格式就回傳 0
    if "萬" in value:
        return int(float(value.replace("萬", "")) * 10000)
    try:
        return int(float(value))
    except:
        return 0

for col in ["like_count", "view_count", "share_count", "repost_count", "reply_count"]:
    df[col] = df[col].apply(parse_count)

# === 布林欄位處理 ===
# 將原始欄位（Y/N）轉換為 True/False
# 處理過程會去除空白、轉成大寫
df["has_photo"] = df["has_photo"].apply(lambda x: str(x).strip().upper() == "Y")
df["has_video"] = df["has_video"].apply(lambda x: str(x).strip().upper() == "Y")

# === emoji 萃取與統計 ===
# 檢查是否為文字型別，如果是文字，從中萃取出所有 emoji 字元並串接成字串回傳
def extract_emojis(text):
    if not isinstance(text, str): return ""
    return "".join([ch for ch in text if ch in emoji.EMOJI_DATA])

df["emojis"] = df["content"].apply(extract_emojis)
df["emoji_count"] = df["emojis"].apply(len)

# # === 語言偵測修正版===
# def detect_lang_custom(text):
#     try:
#         text = str(text)
#         chinese_chars = re.findall(r'[\u4e00-\u9fff]', text)
#         if len(chinese_chars) / max(len(text), 1) > 0.3:
#             return "Ch"
#         return detect(text)
#     except:
#         return "unknown"

# 使用先前定義好的語言偵測函數 detect_lang_with_preprocessing_lingua()，處理每篇文章的語言判定
df["lang"] = df["content"].apply(detect_lang_with_preprocessing_lingua)

# === scrape_time 處理（轉換時區 + 抽取星期與小時）===
# 將時間欄位轉為台北時區，額外抽出格式化後的時間字串、星期幾、小時(0–23）
df["scrape_time_origin"] = pd.to_datetime(df["scrape_time"], utc=True).dt.tz_convert("Asia/Taipei")
df["scrape_time"]  = df["scrape_time_origin"].dt.strftime("%Y年%m月%d日 %H:%M")
df["post_weekday"] = df["scrape_time_origin"].dt.day_name()
df["post_hour"] = df["scrape_time_origin"].dt.hour

# === 是否為高流量文章（破萬）===
# 超過等於 10,000 瀏覽為 1，其餘為 0
df["viral"] = (df["view_count"] >= 10000).astype(int)

# === 是否使用問號、驚嘆號 ===
df["has_question"] = df["content"].apply(lambda x: "？" in str(x) or "?" in str(x))
df["has_exclaim"] = df["content"].apply(lambda x: "！" in str(x) or "!" in str(x))

# === 儲存結果 ===
df.to_csv("threads_cleaned_v1.csv",encoding='utf_8_sig', index=False)
print("✅ 處理完成，已輸出 threads_cleaned_v1.csv")
display(df)


✅ 處理完成，已輸出 threads_cleaned_v1.csv


,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,...,scrape_time,emojis,emoji_count,lang,scrape_time_origin,post_weekday,post_hour,viral,has_question,has_exclaim
0,ayofvr,2025-04-29T10:58:39.000Z,NaN,3小時,Thank you God for another day.,False,False,190,3,23,...,2025年04月30日 06:27,,0,en,2025-04-30 06:27:40.176749+08:00,Wednesday,6,0,False,False
1,NaN,2025-04-29T12:33:44.000Z,NaN,1小時,百達翡麗？ 沒有下限的網路病態！,True,False,196,16,0,...,2025年04月30日 06:27,,0,Ch,2025-04-30 06:27:54.964788+08:00,Wednesday,6,1,True,True
2,ribboworld2021,2025-04-29T04:39:46.000Z,小一日常,9小時,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,False,False,75,6,0,...,2025年04月30日 06:28,,0,Ch,2025-04-30 06:28:09.873641+08:00,Wednesday,6,0,False,True
3,ayofvr,2025-04-29T11:25:22.000Z,NaN,3小時,Just be strong. Confident. Hopeful. Intellectu...,False,False,83,5,3,...,2025年04月30日 06:28,,0,en,2025-04-30 06:28:24.726576+08:00,Wednesday,6,0,False,False
4,jose_ykc,2025-04-29T10:36:19.000Z,輔仁大學,3小時,日文輔系老師上課內容之一AiScReam 歌詞導讀,True,True,4334,55,513,...,2025年04月30日 06:28,,0,Ch,2025-04-30 06:28:39.393706+08:00,Wednesday,6,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,2025年04月29日 07:30,NaN,8小時,First tasting in California,False,False,7,0,0,...,2025年04月30日 00:23,,0,en,2025-04-30 00:23:40.328046+08:00,Wednesday,0,0,False,False
3097,cape__man,2025年04月29日 06:31,NaN,9小時,I hoped it would have been better.,False,False,2,0,0,...,2025年04月30日 00:23,,0,en,2025-04-30 00:23:55.063517+08:00,Wednesday,0,0,False,False
3098,simimoonlight,2025年04月29日 06:26,NaN,9小時,I can’t wait to watch Beyoncé on TikTok tonigh...,False,False,2656,29,280,...,2025年04月30日 00:24,🫶🏿🥹,3,en,2025-04-30 00:24:39.870994+08:00,Wednesday,0,1,False,False
3099,other98,2025年04月29日 12:31,NaN,3小時,"Tonight, Canada just proved that they have a h...",False,False,10000,214,214,...,2025年04月30日 00:24,🙌,1,en,2025-04-30 00:24:54.669936+08:00,Wednesday,0,1,False,True


## 清洗數據V2

In [12]:
# --- 文章長度 ---
df["content_length"] = df["content"].apply(lambda x: len(str(x)))

# --- 是否包含網址 ---
df["has_url"] = df["content"].apply(lambda x: "http" in str(x) or "www." in str(x))

# --- 是否包含 @標記他人 ---
df["has_mention"] = df["content"].apply(lambda x: "@" in str(x))

# --- 是否使用 Hashtag ---
df["has_hashtag"] = df["content"].apply(lambda x: "#" in str(x))

# 貼文主題字詞提取（可後續做 TF-IDF 或主題建模）
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=100, stop_words='english')
word_matrix = vectorizer.fit_transform(df['content'].astype(str))

# 將常見詞語提取出來
keywords = vectorizer.get_feature_names_out()

# 是否為深夜或白天貼文（時間段分類）
def time_period(hour):
    if 5 <= hour < 12:
        return "morning"
    elif 12 <= hour < 17:
        return "afternoon"
    elif 17 <= hour < 22:
        return "evening"
    else:
        return "night"
    
# 判斷是否合法的time_info格式
def is_valid_time_info(text):
    if pd.isna(text):
        return False
    text = str(text).strip()
    return bool(re.match(r"^\d+\s*(分鐘|小時|天)$", text))

df = df[df["time_info"].apply(is_valid_time_info)].copy()

# 將time_info 統一轉為小時
def convert_time_info(text):
    if pd.isna(text):
        return 0
    text = str(text)
    if "分鐘" in text:
        match = re.search(r"(\d+)", text)
        return round(int(match.group(1)) / 60, 1)
    elif "小時" in text:
        match = re.search(r"(\d+)", text)
        return round(int(match.group(1)), 1)
    elif "天" in text:
        match = re.search(r"(\d+)", text)
        return round(int(match.group(1)) * 24, 1)
    elif "週" in text or "禮拜" in text:
        match = re.search(r"(\d+)", text)
        return round(int(match.group(1)) * 7 * 24, 1)
    else:
        return 0


df["time_elapsed_hours"] = df["time_info"].apply(convert_time_info)
df["post_period"] = df["post_hour"].apply(time_period)

cols_to_show_first = ['author', 'content', 'content_length', 'lang', 'scrape_time', 'post_weekday', 'post_hour', 'post_period', 'viral']
df = df[cols_to_show_first + [col for col in df.columns if col not in cols_to_show_first]]
df.to_csv("threads_cleaned_v2.csv",encoding='utf_8_sig',index=False)
print("✅ 處理完成，已輸出 threads_cleaned_v2.csv")
display(df)

✅ 處理完成，已輸出 threads_cleaned_v2.csv


,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,post_url,emojis,emoji_count,scrape_time_origin,has_question,has_exclaim,has_url,has_mention,has_hashtag,time_elapsed_hours
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,https://www.threads.net/@ayofvr/post/DJBymf8uTrK,,0,2025-04-30 06:27:40.176749+08:00,False,False,False,False,False,3.0
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,https://www.threads.net/@ban.mei.onnnnni/post/...,,0,2025-04-30 06:27:54.964788+08:00,True,True,False,False,False,1.0
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,https://www.threads.net/@ribboworld2021/post/D...,,0,2025-04-30 06:28:09.873641+08:00,False,True,False,False,False,9.0
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,https://www.threads.net/@ayofvr/post/DJB1qP5OmzP,,0,2025-04-30 06:28:24.726576+08:00,False,False,False,False,False,3.0
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,https://www.threads.net/@jose_ykc/post/DJBvpGI...,,0,2025-04-30 06:28:39.393706+08:00,False,False,False,False,False,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,https://www.threads.net/@leighton.williams/pos...,,0,2025-04-30 00:23:40.328046+08:00,False,False,False,False,False,8.0
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,https://www.threads.net/@cape__man/post/DJAdFd...,,0,2025-04-30 00:23:55.063517+08:00,False,False,False,False,False,9.0
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,https://www.threads.net/@simimoonlight/post/DJ...,🫶🏿🥹,3,2025-04-30 00:24:39.870994+08:00,False,False,False,False,False,9.0
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,https://www.threads.net/@other98/post/DJBGV3NxiX_,🙌,1,2025-04-30 00:24:54.669936+08:00,False,True,False,False,False,3.0


## Self-Attention 模組處理文字資料（測試中 因未寫完可先跳過）

In [6]:
# BERT 會把 [EMOJI], [CONTENT] 當作分界的詞來理解
def build_semantic_text(row):
    parts = []

    if pd.notna(row["author"]):
        parts.append(f"[AUTHOR] {row['author']}")

    if pd.notna(row["topic"]):
        parts.append(f"[TOPIC] {row['topic']}")

    if pd.notna(row["emojis"]):
        emoji_text = " ".join(row["emojis"])
        parts.append(f"[EMOJI] {emoji_text}")

    if pd.notna(row["content"]):
        parts.append(f"[CONTENT] {row['content']}")

    return " ".join(parts)


# 建立 semantic_text，不影響 df 本身
semantic_text_series = df.apply(build_semantic_text, axis=1)

# 若你想要新 DataFrame：
df_semantic = df.copy()
df_semantic["semantic_text"] = semantic_text_series
df_semantic


,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,emojis,emoji_count,scrape_time_origin,has_question,has_exclaim,has_url,has_mention,has_hashtag,time_elapsed_hours,semantic_text
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,,0,2025-04-30 06:27:40.176749+08:00,False,False,False,False,False,3.0,[AUTHOR] ayofvr [EMOJI] [CONTENT] Thank you G...
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,,0,2025-04-30 06:27:54.964788+08:00,True,True,False,False,False,1.0,[EMOJI] [CONTENT] 百達翡麗？ 沒有下限的網路病態！
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,,0,2025-04-30 06:28:09.873641+08:00,False,True,False,False,False,9.0,[AUTHOR] ribboworld2021 [TOPIC] 小一日常 [EMOJI] ...
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,,0,2025-04-30 06:28:24.726576+08:00,False,False,False,False,False,3.0,[AUTHOR] ayofvr [EMOJI] [CONTENT] Just be str...
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,,0,2025-04-30 06:28:39.393706+08:00,False,False,False,False,False,3.0,[AUTHOR] jose_ykc [TOPIC] 輔仁大學 [EMOJI] [CONTE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,,0,2025-04-30 00:23:40.328046+08:00,False,False,False,False,False,8.0,[AUTHOR] leighton.williams [EMOJI] [CONTENT] ...
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,,0,2025-04-30 00:23:55.063517+08:00,False,False,False,False,False,9.0,[AUTHOR] cape__man [EMOJI] [CONTENT] I hoped ...
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,🫶🏿🥹,3,2025-04-30 00:24:39.870994+08:00,False,False,False,False,False,9.0,[AUTHOR] simimoonlight [EMOJI] 🫶 🏿 🥹 [CONTENT]...
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,🙌,1,2025-04-30 00:24:54.669936+08:00,False,True,False,False,False,3.0,"[AUTHOR] other98 [EMOJI] 🙌 [CONTENT] Tonight, ..."


In [7]:
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import Dataset, DataLoader

# 載入 BERT tokenizer 和模型
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = BertModel.from_pretrained("bert-base-chinese")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 自訂 Dataset 類別（不變）
class TextDataset(Dataset):
    def __init__(self, texts, max_len=128):
        self.encodings = tokenizer(
            texts, 
            truncation=True,
            padding='max_length',
            max_length=max_len,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx]
        }

# 建立 dataset 和 dataloader（這裡建議用 semantic_text 而不是 content）
texts = df_semantic["semantic_text"].astype(str).tolist()
text_dataset = TextDataset(texts)
text_loader = DataLoader(text_dataset, batch_size=32)

# 提取語意向量 Z_text
Z_text_list = []

model.eval()
with torch.no_grad():
    for batch in text_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        cls_rep = outputs.last_hidden_state[:, 0, :]  # 取 [CLS] 向量
        
        Z_text_list.append(cls_rep.cpu())

Z_text_tensor = torch.cat(Z_text_list, dim=0)  # [N, 768]



In [8]:
import pandas as pd

Z_text_df = pd.DataFrame(Z_text_tensor.numpy())
Z_text_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-1.197812,0.491429,-0.485847,0.316401,1.326630,-1.078312,0.297176,-0.804608,-1.177580,0.156056,...,-0.562810,-0.411340,0.433298,-0.422068,-0.669982,-0.726443,0.106795,1.173453,0.316311,0.108925
1,0.215056,1.000118,-0.044262,-0.209858,0.740916,-1.110934,-0.698522,-0.050963,-0.216489,1.010718,...,0.426817,-0.219904,-0.014254,-0.705548,0.285813,-0.596890,-0.212060,0.695832,-0.082648,0.383929
2,0.887392,0.952374,0.468288,0.363257,1.021098,-1.253425,-0.300586,-0.201750,-0.855971,0.126374,...,0.325378,-0.584822,-0.845940,-0.637423,-0.245427,0.500610,0.191019,0.498423,0.164274,-0.312768
3,-1.188987,0.500823,-1.228274,-0.300035,1.055788,-1.166170,0.346816,-0.645881,-0.857121,0.474149,...,-0.987927,-0.638470,-0.161924,-0.339970,-0.626445,-0.612481,0.194379,1.441311,-0.297321,0.020321
4,-0.932422,0.218099,-1.037628,0.035938,1.054755,-0.978883,-0.409518,0.187629,-0.958486,0.335753,...,-0.653077,-0.492126,0.483244,-0.070685,-0.069700,0.078444,0.408898,1.241559,-0.151853,-0.196577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,-1.150749,0.211006,-0.857209,0.486619,0.887699,-1.112387,0.242534,-0.489165,-1.174777,0.081529,...,-0.239787,-0.378836,-0.029298,-0.404599,-0.395586,-0.504007,-0.001980,1.394140,-0.356526,-0.057205
3079,-0.742924,-0.134855,-0.337486,0.413057,1.205682,-1.366358,-0.096314,-0.656512,-1.172672,0.480091,...,-0.972684,-0.425606,0.257879,-0.083000,-0.208448,-0.519165,0.292847,1.117920,0.169528,0.234989
3080,-1.257781,0.298899,-0.860130,0.276141,1.046705,-0.745504,-0.342909,-0.462435,-1.735537,0.148792,...,-0.665857,-0.498027,-0.127219,-1.029894,-0.855351,-0.247067,0.405077,1.160195,0.243745,0.261609
3081,-0.801286,0.418368,-0.866499,-0.064372,0.974501,-1.084798,0.472064,-0.299250,-0.953439,0.027430,...,-1.174034,-0.653110,0.092794,-0.495103,-0.692927,-0.327008,0.045146,1.080402,0.078887,-0.075638


## MLP 處理數值資料

In [9]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

# 假設你已載入 DataFrame 為 df，並有這些欄位
numeric_cols = [
    "view_count", "followers_count", "emoji_count", "content_length", "post_hour", "time_elapsed_hours",
    "has_photo", "like_count", "reply_count", "repost_count", "share_count",
    "has_hashtag", "has_url", "has_mention", "has_exclaim", "has_question"
]

# 布林欄位轉 int（保險起見）
for col in numeric_cols:
    if df_semantic[col].dtype == bool:
        df_semantic[col] = df_semantic[col].astype(int)

# 標準化數值特徵
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(df_semantic[numeric_cols])

# 定義簡單的 MLP 模型
class NumericMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# 初始化與執行模型
# 自動偵測你是否有 GPU（用 CUDA），否則就用 CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
# 根據你數值欄位的數量設定 input_dim
model_num = NumericMLP(input_dim=X_num_scaled.shape[1])
# 把模型送到對應裝置（CPU / GPU）
model_num.to(device)

# 暫時關閉梯度運算，因沒有要訓練模型，僅讓資料過神經網路
with torch.no_grad():
    X_tensor = torch.tensor(X_num_scaled, dtype=torch.float32).to(device)
    Z_num_tensor = model_num(X_tensor).cpu()

# 合併 Z_text 和 Z_num
Z_text_tensor = torch.tensor(Z_text_df.values, dtype=torch.float32)  # [N, 768]
Z_full_tensor = torch.cat([Z_text_tensor, Z_num_tensor], dim=1)      # [N, 896]

print("數值向量與語意向量已成功融合為 Z_full_tensor")
Z_full_tensor

數值向量與語意向量已成功融合為 Z_full_tensor


tensor([[-1.1978,  0.4914, -0.4858,  ..., -0.0843, -0.0798,  0.0808],
        [ 0.2151,  1.0001, -0.0443,  ...,  0.3707, -0.0328, -0.3490],
        [ 0.8874,  0.9524,  0.4683,  ...,  0.1201,  0.1975, -0.1585],
        ...,
        [-1.2578,  0.2989, -0.8601,  ...,  0.0864,  0.0967, -0.2644],
        [-0.8013,  0.4184, -0.8665,  ...,  0.5446,  0.1440, -0.5102],
        [-0.7080,  0.6756, -0.2339,  ...,  0.0632,  0.0072,  0.0233]])

## 清洗數據embbedding

In [10]:
from datasets import Dataset

# =============== BERT 向量嵌入 ===============
df = df.dropna(subset=['content']) #要先處理content空值才能embedding
# --- 載入 tokenizer & model ---
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")
# 選擇硬體設備（MPS、CUDA、CPU），自動判斷是否可用 GPU（M1/M2 晶片上的 MPS 或 CUDA），否則 fallback 到 CPU
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# --- 建立 HuggingFace Dataset ---
hf_dataset = Dataset.from_pandas(df[["content"]])

# --- tokenize function ---
def tokenize_function(examples):
    return tokenizer(examples['content'], truncation=True, padding='max_length', max_length=128)
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# --- 取得 [CLS] 向量 ---
def extract_embeddings(batch):
    inputs = {k: torch.tensor(v).to(model.device) for k, v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()
    return {"embeddings": embeddings}

# --- 批次轉換為 embeddings ---
batch_size = 64
embeddings_dataset = tokenized_dataset.map(extract_embeddings, batched=True, batch_size=batch_size)

# =============== 匯出最終結果 ===============
# embeddings_dataset["embeddings"] 是 list of 768-dim vectors
embedding_df = pd.DataFrame(embeddings_dataset["embeddings"])
final_df = pd.concat([df.reset_index(drop=True), embedding_df], axis=1)

# 儲存
# final_df.to_csv("C:/Users/User/Desktop/louis/threads_with_embeddings.csv",encoding='utf_8_sig', index=False)
final_df.to_csv("threads_with_embeddings.csv",encoding='utf_8_sig', index=False)
print("✅ 全部處理完成，已輸出 threads_with_embeddings.csv")

Map: 100%|██████████| 3083/3083 [01:23<00:00, 37.14 examples/s]


✅ 全部處理完成，已輸出 threads_with_embeddings.csv


## 分詞

In [15]:
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

def tokenize_and_remove_stopwords(text):
    if not isinstance(text, str):
        return ''
    words = jieba.cut(text)
    words_filtered = [word for word in words if word not in stopwords]
    return ' '.join(words_filtered)


df['processed_content'] = df['content'].apply(tokenize_and_remove_stopwords)
df['processed_content'][2]

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/59/tx1m33l955b0h85n6dlwm4440000gn/T/jieba.cache
Loading model cost 0.499 seconds.
Prefix dict has been built successfully.


'考完 期中考 ， 成績 都 還沒出 來 ， 小一 女兒 就 自信 對 我 說 ： 「 我 真羨慕 妳 生 一個 天才 ！ 」'

## 機器學習建模

In [16]:
# 計算 TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_content'])

# 計算 TF
tf_vectorizer = CountVectorizer()
tf_matrix = tf_vectorizer.fit_transform(df['processed_content'])

print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 33538 stored elements and shape (3083, 12873)>
  Coords	Values
  (0, 2799)	0.4663056843152708
  (0, 3188)	0.2940455207159238
  (0, 1306)	0.44284097627197305
  (0, 1210)	0.31028579706458703
  (0, 412)	0.503121337625026
  (0, 892)	0.3880460720366238
  (1, 9695)	0.44774054192975266
  (1, 10472)	0.44774054192975266
  (1, 8813)	0.2568875443528821
  (1, 3491)	0.42597495668833046
  (1, 10367)	0.38876644295524665
  (1, 9613)	0.44774054192975266
  (2, 10514)	0.31144203947399207
  (2, 8415)	0.3288785193471119
  (2, 7290)	0.28537598711095685
  (2, 12071)	0.35868637857163266
  (2, 6458)	0.35868637857163266
  (2, 6044)	0.2990706601225911
  (2, 10681)	0.3288785193471119
  (2, 9846)	0.35868637857163266
  (2, 3221)	0.18120453315645962
  (2, 5960)	0.31144203947399207
  (3, 1628)	0.26164267502394156
  (3, 538)	0.23056465830230907
  (3, 2699)	0.3728698840922293
  :	:
  (3080, 3034)	0.4140307205152917
  (3080, 3064)	0.38762922360836444
  (3080,

# 多模型分類實驗

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, random_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 8
num_epochs = 5
# ========== 參數設定 ==========
model_tokenizer_map = {
    "FusionMacBERT": "hfl/chinese-macbert-base",
    "PureMacBERT": "hfl/chinese-macbert-base",
    "NumericOnly": None,
    "BiLSTMWithNumeric": "bert-base-chinese",
    "MacBERTWithGRU": "hfl/chinese-macbert-base",
    "MacBERTMLPFusion": "hfl/chinese-macbert-base",
    "TextCNNMacBERT": "hfl/chinese-macbert-base",
    "RoBERTa": "hfl/chinese-roberta-wwm-ext",
    "BERTwwmExt": "hfl/chinese-bert-wwm-ext",
    "ERNIE": "nghuyong/ernie-3.0-base-zh",
    "ConvBERT": "YituTech/conv-bert-base"
}

#tokenizer
default_tokenizer_name = model_tokenizer_map["FusionMacBERT"]
tokenizer = AutoTokenizer.from_pretrained(default_tokenizer_name)

#載入資料
# df = pd.read_csv("C:/Users/User/Desktop/louis/threads_cleaned_v2.csv", encoding='utf_8_sig')
#df = df.dropna(subset=['content', 'view_count']).reset_index(drop=True)
df

,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,emojis,emoji_count,scrape_time_origin,has_question,has_exclaim,has_url,has_mention,has_hashtag,time_elapsed_hours,processed_content
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,,0,2025-04-30 06:27:40.176749+08:00,False,False,False,False,False,3.0,Thank you God for another day .
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,,0,2025-04-30 06:27:54.964788+08:00,True,True,False,False,False,1.0,百達 翡麗 ？ 沒有 下限 網路 病態 ！
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,,0,2025-04-30 06:28:09.873641+08:00,False,True,False,False,False,9.0,考完 期中考 ， 成績 都 還沒出 來 ， 小一 女兒 就 自信 對 我 說 ： 「 我 真...
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,,0,2025-04-30 06:28:24.726576+08:00,False,False,False,False,False,3.0,Just be strong . Confident . Hopeful ....
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,,0,2025-04-30 06:28:39.393706+08:00,False,False,False,False,False,3.0,日文 輔系 老師 上 課內容 之一 AiScReam 歌詞 導讀
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,,0,2025-04-30 00:23:40.328046+08:00,False,False,False,False,False,8.0,First tasting in California
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,,0,2025-04-30 00:23:55.063517+08:00,False,False,False,False,False,9.0,I hoped it would have been better .
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,🫶🏿🥹,3,2025-04-30 00:24:39.870994+08:00,False,False,False,False,False,9.0,I can ’ t wait to watch Beyonc é o...
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,🙌,1,2025-04-30 00:24:54.669936+08:00,False,True,False,False,False,3.0,"Tonight , Canada just proved that th..."


## Label 分群 （1000以下、1000~10000、10000~100000、100000以上)

In [22]:
# 將 view_count 分成四類：
# 0: 小於 1000
# 1: 1000 ~ 9999
# 2: 10000 ~ 99999
# 3: 100000 以上

def map_view_class(x):
    if x < 1000:
        return 'low'
    elif x < 10000:
        return 'medium'
    elif x < 100000:
        return 'high'
    else:
        return 'very_high'

df['view_class'] = df['view_count'].apply(map_view_class)

# 編碼成數字 label
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['view_class'])

df_high = df[df['view_class'] == 'very_high']
df_medium = df[df['view_class'] == 'high']
df_low = df[df['view_class'] == 'medium']
df_very_low = df[df['view_class'] == 'low']

# 針對較少的類別進行擴增（假設 high 和 very_low 比較少）
df_high_oversampled = pd.concat([df_high] * 3, ignore_index=True)
df_very_low_oversampled = pd.concat([df_very_low] * 3, ignore_index=True)

# 合併並打亂
df = pd.concat([df_medium, df_low, df_high_oversampled, df_very_low_oversampled], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

df

,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,scrape_time_origin,has_question,has_exclaim,has_url,has_mention,has_hashtag,time_elapsed_hours,processed_content,view_class,label
0,_jili.25,我要讓掉原本的才能去vip啊啊啊啊😭😭 1,21,Ch,2025年04月30日 08:28,Wednesday,8,morning,0,2025-04-29T06:48:05.000Z,...,2025-04-30 08:28:10.692106+08:00,False,False,False,False,False,9.0,我要 讓 掉 原本 才能 去 vip 啊啊啊 啊 😭 😭 1,low,1
1,lv_auer,拔完智齒第三天了，到底要痛幾天，我都快不能吃東西了😂😂 #哦藉機減肥,34,Ch,2025年04月30日 08:07,Wednesday,8,morning,0,2025-04-29T10:55:49.000Z,...,2025-04-30 08:07:36.828332+08:00,False,False,False,False,True,5.0,拔 完智齒 第三天 ， 到底 要痛 幾天 ， 我 都 快 不能 吃 東西 😂 😂 # 哦...,low,1
2,cai.7052,如果有下輩子的話，我一定要當女生，當男生太累太苦了,25,Ch,2025年05月03日 19:30,Saturday,19,evening,0,2025-05-03T01:07:29.000Z,...,2025-05-03 19:30:28.096090+08:00,False,False,False,False,False,2.0,如果 下輩子 話 ， 我 一定 要當 女生 ， 當 男生 太累 太苦,low,1
3,brandin.podziemski2,拍得我真帥,5,Ch,2025年05月03日 19:55,Saturday,19,evening,0,2025-05-03T00:07:39.000Z,...,2025-05-03 19:55:44.483391+08:00,False,False,False,False,False,3.0,拍得 我 真帥,low,1
4,emo_97_10_31,🫤在脆上面真的可以找到真心的朋友嗎,17,Ch,2025年04月30日 07:50,Wednesday,7,morning,0,2025-04-29T11:39:04.000Z,...,2025-04-30 07:50:25.230620+08:00,False,False,False,False,False,4.0,🫤 脆 上面 真的 可以 找到 真心 朋友 嗎,low,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11342,goooosleep,为什么蝴蝶要飞到十六楼？,12,Ch,2025年04月30日 00:08,Wednesday,0,night,0,2025年04月29日 12:03,...,2025-04-30 00:08:40.964074+08:00,True,False,False,False,False,4.0,为什么 蝴蝶 要 飞到 十六 楼 ？,low,1
11343,kickflip_said,我現在看到大家買到票還有互報說VIP區還有票的串文就是一直在破防 辦在期末考前一週我真的要哭出來,48,Ch,2025年04月29日 07:24,Tuesday,7,morning,0,2025年04月28日 18:47,...,2025-04-29 07:24:48.150110+08:00,False,False,False,False,False,4.0,我現 看到 大家 買到 票 還有 互報 說 VIP 區還 有票 串文 就是 一直 破防 ...,low,1
11344,lucasbiubiubiubiu,京东外卖真的是一次都没准时过，不是早半个小时就是晚半个小时😡,30,Ch,2025年04月29日 07:14,Tuesday,7,morning,0,2025年04月28日 11:27,...,2025-04-29 07:14:37.311967+08:00,False,False,False,False,False,11.0,京东 外卖 真的 一次 都 没准 时过 ， 不是 早 半个 小时 就是 晚 半个 小时 😡,low,1
11345,clydecole620,Malik Beasley. Michael Ealy. Pop a wheelie. Kn...,56,tl,2025年05月03日 08:00,Saturday,8,morning,0,2025-05-02T15:29:08.000Z,...,2025-05-03 08:00:53.883370+08:00,False,False,False,False,False,0.5,Malik Beasley . Michael Ealy . Pop a...,low,1


# Normalization 數值特徵標準化

In [25]:
# Normalization 數值特徵標準化
base_num_cols = ['like_count', 'share_count', 'repost_count', 'reply_count', 'emoji_count', 'has_photo', 'has_video', 'has_question', 'has_exclaim', 'has_mention', 'has_url', 'has_hashtag', 'content_length']
# 找出 one-hot 編碼的欄位（語言類型、發文時段、星期幾等類別欄位）
# 使用 StandardScaler 將數值欄位轉換為「標準常態分布」（mean=0, std=1），有助於模型學習穩定。
onehot_cols = [col for col in df.columns if col.startswith('lang_') or col.startswith('post_period_') or col.startswith('post_weekday_')]
num_cols = base_num_cols + onehot_cols
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Tokenizer 文本編碼 ：使用事先定義好的 tokenizer（例如 MacBERT、RoBERTa）對貼文進行斷詞、編碼
# 將編碼後的結果儲存到 df 中，這兩個欄位會作為 BERT 模型的輸入
encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
df['input_ids'] = encodings['input_ids']  # 斷詞後對應的詞彙 ID
df['attention_mask'] = encodings['attention_mask']  # 對應位置是否是 padding（0）或實際內容（1）

In [26]:
class RegressionDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.numerics = df[num_cols].values
        self.targets = df['target'].values

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'target': torch.tensor(self.targets[idx], dtype=torch.float)
        }


In [ ]:
df

In [27]:
# Dataset
class CustomDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['label'].tolist()
        self.numerics = df[num_cols].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

#模型架構

In [28]:
#模型架構
# 1. FusionMacBERT：BERT + 數值特徵 concat
class FusionMacBERTModel(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(self.bert.config.hidden_size + 64, num_classes)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 2. PureMacBERT：只有文字
class PureMacBERTModel(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, numerics=None):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        return self.classifier(self.dropout(cls_output))

# 3. NumericOnly：只有數值特徵
class NumericOnlyModel(nn.Module):
    def __init__(self, num_numeric_features, num_classes):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(num_numeric_features, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, input_ids=None, attention_mask=None, numerics=None):
        return self.classifier(numerics)

# 4. BiLSTMWithNumeric：LSTM 處理詞嵌入 + 數值特徵
class BiLSTMWithNumeric(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_numeric_features, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden_dim * 2 + 64, num_classes)

    def forward(self, input_ids, attention_mask, numerics):
        x = self.embedding(input_ids)
        lstm_out, _ = self.lstm(x)
        pooled = lstm_out[:, -1, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((pooled, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 5. MacBERTWithGRU：BERT + GRU + 數值特徵
class MacBERTWithGRU(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.gru = nn.GRU(self.bert.config.hidden_size, 128, batch_first=True, bidirectional=True)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(128*2 + 64, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask, numerics):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        gru_out, _ = self.gru(bert_out)
        pooled = gru_out[:, -1, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((pooled, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 6. MacBERTMLPFusion：BERT + 數值特徵 -> MLP
class MacBERTMLPFusion(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.fc = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size + num_numeric_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        combined = torch.cat((cls_output, numerics), dim=1)
        return self.fc(combined)

# 7. TextCNNMacBERT：BERT 輸出卷積後 + 數值特徵
class TextCNNMacBERT(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.convs = nn.ModuleList([nn.Conv2d(1, 64, (k, self.bert.config.hidden_size)) for k in [2, 3, 4]])
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(64 * len([2, 3, 4]) + 64, num_classes)

    def conv_and_pool(self, x, conv):
        x = torch.relu(conv(x)).squeeze(3)
        x = torch.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, input_ids, attention_mask, numerics):
        x = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state.unsqueeze(1)
        cnn_out = torch.cat([self.conv_and_pool(x, conv) for conv in self.convs], 1)
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cnn_out, num_out), dim=1)
        return self.classifier(combined)


In [37]:
#訓練與評估
def train_and_eval(model, name, preview_count=10):
    model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

    loss_fn = FocalLoss()
    # loss_fn = nn.MSELoss()

    # 訓練階段
    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()

    # 評估階段
    model.eval()
    all_preds, all_labels = [], []
    '''
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            preds = output.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    print(f"[{name} 評估結果] MSE: {mse:.2f} | MAE: {mae:.2f}")
    print(f"\n{name} 評估結果：")
    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))
    '''
    preview_shown = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            preds = output.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

            #印出前幾筆的預測、真實值
            if preview_shown < preview_count:
                batch_size = input_ids.shape[0]
                for i in range(batch_size):
                    if preview_shown >= preview_count:
                        break
                    input_id = input_ids[i].cpu().numpy()
                    text = tokenizer.decode(input_id, skip_special_tokens=True)
                    print(f"\n[{name} 預測] 第 {preview_shown+1} 筆")
                    print(f"Text: {text}")
                    print(f"Predicted: {label_encoder.inverse_transform([preds[i]])[0]}")
                    print(f"Actual:    {label_encoder.inverse_transform([labels[i].cpu().item()])[0]}")
                    preview_shown += 1

    print(f"\n{name} 評估結果：")
    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

In [ ]:
# 資料分割：資料集切分與取樣
dataset = CustomDataset(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset)-train_size])
train_labels = [train_dataset[i]['labels'].item() for i in range(len(train_dataset))]
class_counts = pd.Series(train_labels).value_counts().to_dict()
weights = [1.0 / class_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=8, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=8)

# 執行多模型訓練
model_variants = {
    # "FusionMacRegressor": FusionMacBERTRegressor("hfl/chinese-macbert-base", len(num_cols), 3),
    "FusionMacBERT": FusionMacBERTModel("hfl/chinese-macbert-base", len(num_cols), 4),
    "PureMacBERT": PureMacBERTModel("hfl/chinese-macbert-base", 4),
    "NumericOnly": NumericOnlyModel(len(num_cols), 4),
    "BiLSTMWithNumeric": BiLSTMWithNumeric(tokenizer.vocab_size, 128, 128, len(num_cols), 4),
    "MacBERTWithGRU": MacBERTWithGRU("hfl/chinese-macbert-base", len(num_cols), 4),
    "MacBERTMLPFusion": MacBERTMLPFusion("hfl/chinese-macbert-base", len(num_cols), 4),
    "TextCNNMacBERT": TextCNNMacBERT("hfl/chinese-macbert-base", len(num_cols), 4),
    "RoBERTa": FusionMacBERTModel("hfl/chinese-roberta-wwm-ext", len(num_cols), 4),
    "BERTwwmExt": FusionMacBERTModel("hfl/chinese-bert-wwm-ext", len(num_cols), 4),
    "ERNIE": FusionMacBERTModel("nghuyong/ernie-3.0-base-zh", len(num_cols), 4),
    "ConvBERT": FusionMacBERTModel("YituTech/conv-bert-base", len(num_cols), 4)
}

# 逐個模型訓練與輸出結果
for name, model in model_variants.items():
    tokenizer_name = model_tokenizer_map.get(name, default_tokenizer_name)
    if tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
        df['input_ids'] = encodings['input_ids']
        df['attention_mask'] = encodings['attention_mask']
    train_and_eval(model, name)

1. FusionMacBERT ✅文字 + ✅數值
BERT： 使用 MacBERT
架構： 把 [CLS] 向量與數值特徵經過 MLP 融合
用途： 做為 baseline 融合模型
優點： 同時考慮內容語義與貼文統計資料（如按讚數、是否有 hashtag）

2. PureMacBERT ✅文字 + ❌數值
BERT： 使用 MacBERT
架構： 單純使用 [CLS]，後接 linear 層分類
用途： 純語言模型 baseline
對照： 可用來比較是否有數值輔助提升效果

3. NumericOnly ❌文字 + ✅數值
模型類型： 只有數值輸入，經過 MLP 做分類
用途： 測試「只靠貼文統計資料」能否達到合理分類
對照： 可與文字模型或融合模型對比效果

4. BiLSTMWithNumeric ✅文字（Embedding+LSTM）+ ✅數值
嵌入方式： 使用 nn.Embedding + BiLSTM 處理文字（不是 BERT）
融合方式： 將 LSTM 最後時間步 + 數值特徵拼接
特別點： 測試「非 Transformer 模型」是否仍具競爭力

5. MacBERTWithGRU ✅文字（MacBERT）+ ✅數值
文字處理： MacBERT 之後再串 GRU
融合方式： GRU 輸出最後一步拼接數值特徵
意圖： 想看 BERT+RNN 的表現 vs. 傳統 BERT

6. MacBERTMLPFusion ✅文字 + ✅數值
處理方式： 文字與數值直接拼接後進入 MLP
不同於 FusionMacBERT：
沒有額外處理數值特徵（如沒有經過 nn.Linear)
更單純的融合設計（屬於 Early Fusion）

7. TextCNNMacBERT ✅文字 + ✅數值
模型組合：
使用 BERT 編碼後丟進 CNN filter (TextCNN)
再與數值特徵融合
用途： 測試 BERT 結合 CNN 特徵提取是否提升效果
有趣點： 有些短文模型（如微博、Threads）對 CNN 特徵抓取敏感

8. RoBERTa ✅文字 + ✅數值
BERT 替代品： 改用 RoBERTa（中文版本）
融合方式： 同 FusionMacBERT
實驗目的： 測試不同語言模型對結果的影響（語言模型 ablation）


9. BERTwwmExt ✅文字 + ✅數值
BERT： 使用 Chinese BERT whole-word-masking 擴展版
比較目的： 同上，用於測試不同語言模型特性的影響

10. ERNIE ✅文字 + ✅數值
BERT： 改用百度的 ERNIE（引入知識增強）
適用場景： 當文本與常識有關（如話題、用語）
目的： 評估知識型語言模型在社群文本分類的效果

11. ConvBERT ✅文字 + ✅數值
模型特色： 使用 Convolution + Self-Attention 混合架構的 BERT
實驗意義： 試驗非傳統 Self-Attention 模型是否有優勢


| 模型名稱              | 說明               | 是否融合 | 文本處理法         | 特殊處理       |
| ----------------- | ---------------- | ---- | ------------- | ---------- |
| FusionMacBERT     | BERT + 數值特徵      | ✅    | MacBERT       | 自製融合層      |
| PureMacBERT       | 純文本模型            | ❌    | MacBERT       | baseline   |
| NumericOnly       | 純統計數值            | ❌    | 無             | MLP only   |
| BiLSTMWithNumeric | LSTM + 數值        | ✅    | nn.Embedding  | 不使用 BERT   |
| MacBERTWithGRU    | BERT + GRU + 數值  | ✅    | MacBERT + GRU | 時序特徵強化     |
| MacBERTMLPFusion  | BERT + 數值        | ✅    | MacBERT       | 拼接後進 MLP   |
| TextCNNMacBERT    | BERT + CNN + 數值  | ✅    | MacBERT + CNN | 模仿 TextCNN |
| RoBERTa           | 換 BERT backbone  | ✅    | RoBERTa       | 模型比較       |
| BERTwwmExt        | 換 BERT backbone  | ✅    | BERT-wwm      | 模型比較       |
| ERNIE             | 引入知識的 BERT       | ✅    | ERNIE         | 模型比較       |
| ConvBERT          | 混合卷積 + 注意力的 BERT | ✅    | ConvBERT      | 模型比較       |




# Shapley value analysis

In [ ]:
import shap
import torch
import numpy as np
import matplotlib.pyplot as plt

def analyze_numeric_shap(model, df, num_cols, sample_count=200, background_count=100):
    model.eval().cpu()
    X = df[num_cols].values.astype(np.float32)

    def model_forward(x_numpy):
        x_tensor = torch.tensor(x_numpy, dtype=torch.float32)
        with torch.no_grad():
            outputs = model(numerics=x_tensor)
            return outputs.numpy()

    explainer = shap.Explainer(model_forward, shap.sample(X, background_count))
    shap_values = explainer(X[:sample_count])

    print("🔍 特徵平均貢獻值 (bar chart):")
    shap.plots.bar(shap_values)

    print("🐝 特徵影響分布 (beeswarm plot):")
    shap.plots.beeswarm(shap_values)

    return shap_values


In [ ]:
class NumericBranchWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.numeric_fc = model.numeric_fc
        self.classifier = model.classifier
        self.hidden_size = model.bert.config.hidden_size
        self.dummy_bert = torch.zeros((1, self.hidden_size))  # 假裝 CLS 向量是 0

    def forward(self, numerics):
        num_out = torch.relu(self.numeric_fc(numerics))
        dummy_bert = self.dummy_bert.expand(numerics.size(0), -1)
        combined = torch.cat((dummy_bert, num_out), dim=1)
        return self.classifier(combined)

In [ ]:
# ✅ 指定支援 SHAP 的模型名稱清單
shap_supported_models = ["NumericOnly", "FusionMacBERT", "MacBERTMLPFusion"]

for name, model in model_variants.items():
    if name in shap_supported_models:
        print(f"\n📊 分析模型：{name}")

        # 如果是需要包裝的 MacBERT 融合類型
        if name in ["FusionMacBERT", "MacBERTMLPFusion"]:
            wrapper_model = NumericBranchWrapper(model)
            shap_vals = analyze_numeric_shap(wrapper_model, df, num_cols)
        else:
            # 否則就是 NumericOnly 可以直接用
            shap_vals = analyze_numeric_shap(model, df, num_cols)

        # ✅（選配）儲存圖表：如需存圖檔可解除註解
        """
        plt.figure()
        shap.plots.bar(shap_vals, show=False)
        plt.savefig(f"shap_bar_{name}.png", bbox_inches='tight')
        plt.close()

        plt.figure()
        shap.plots.beeswarm(shap_vals, show=False)
        plt.savefig(f"shap_beeswarm_{name}.png", bbox_inches='tight')
        plt.close()
        """


# 迴歸預測

In [14]:
# 原本這樣分類（要拿掉）
# df['view_class'] = ...
# df['label'] = ...

# 直接用原始 view_count 作為 regression target
df = df.dropna(subset=["content", "view_count"])
df['target'] = df['view_count'].apply(parse_count)  # 如果 view_count 不是數字要先轉換


In [15]:
def train_and_eval_regression(model, name):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.MSELoss()

    # 訓練
    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)  # 重要：labels 必須是 float
            preds = model(input_ids, attention_mask, numerics)
            loss = loss_fn(preds, targets)
            loss.backward()
            optimizer.step()

    # 評估
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)
            preds = model(input_ids, attention_mask, numerics)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)
    print(f"\n{name}  MSE: {mse:.2f} | MAE: {mae:.2f}| R2: {r2:.2f}")

    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    return all_targets, all_preds





In [16]:
class FusionMacBERTRegressor(nn.Module):
    def __init__(self, model_name, num_numeric_features, output_dim=1):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.regressor = nn.Linear(self.bert.config.hidden_size + 64, output_dim)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.regressor(self.dropout(combined)).squeeze(1)  # (batch,)


In [17]:
rm -rf ~/.cache/huggingface/transformers/hfl__chinese-macbert-base


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
# Dataset
'''
class CustomDatasetRegression(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['label'].astype(float).values   # ← 為回歸任務需轉成 float
        self.numerics = df[num_cols].astype(float).values  # ← 確保為 float array

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),  # ← 修正為 float
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)       # ← 修正為 float
        }
'''
class CustomDatasetRegression(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['target'].astype(float).values
        self.numerics = df[num_cols].astype(float).values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

# Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [19]:

dataset = CustomDatasetRegression(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)


class FusionMacBERTRegressor(nn.Module):
    def __init__(self, model_name, num_numeric_features, output_dim=1):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.regressor = nn.Linear(self.bert.config.hidden_size + 64, output_dim)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.regressor(self.dropout(combined)).squeeze(1)


def train_and_eval_regression(model, name):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.MSELoss()

    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)

            preds = model(input_ids, attention_mask, numerics)
            loss = loss_fn(preds, targets)
            loss.backward()
            optimizer.step()

    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)

            preds = model(input_ids, attention_mask, numerics)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)

    print(f"\n{name}  MSE: {mse:.2f} | MAE: {mae:.2f} | R²: {r2:.2f}")
    return all_targets, all_preds


model = FusionMacBERTRegressor("hfl/chinese-macbert-base", len(num_cols))
all_targets, all_preds = train_and_eval_regression(model, "FusionMacBERTRegressor")


FusionMacBERTRegressor  MSE: 4410895810.43 | MAE: 10676.62 | R²: -0.03


#**下面都是舊的東西而已~~~~**


In [ ]:
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

def tokenize_and_remove_stopwords(text):
    words = jieba.cut(text)
    words_filtered = [word for word in words if word not in stopwords]
    return ' '.join(words_filtered)

df['processed_content'] = df['Content'].apply(tokenize_and_remove_stopwords)

new_article = "IC 設計大廠聯發科 (2454-TW) 副董事長暨執行長蔡力行今 (26) 日獲頒潘文淵獎，會後受訪表示，聯發科 3 奈米會在台積電 (2330-TW)(TSM-US) 做，且由於先進製程技術相當複雜，不論要採用或更換都非常困難，雙方會持續緊密合作。外界今日提問不論是輝達 (NVDA-US)、蘋果 (AAPL-US) 等都表示尋求多元的晶圓代工方案，蔡力行回應，聯發科在先進製程持續與台積電緊密合作，英特爾 (INTC-US) 則負責 16 奈米蔡力行也強調，聯發科不會只停在採用 4 奈米，也會採用 3 奈米製程，此外，由於電晶體微縮速度趨緩，儘管技術上可行，但不一定符合經濟效益，因此技術也逐步從平面變成 2D、2.5D，甚至 3D 等，先進封裝的重要性比以前增加。至於跟輝達合作，蔡力行重申，雙方合作仍以汽車為主，輝達布局車用比聯發科早，主要著墨在智慧座艙與 ADAS 系統，雙方有很好的配合，其中，輝達主攻高階、聯發科則瞄準中階，雙方正密切合作開會。"
processed_new_article = tokenize_and_remove_stopwords(new_article)

# Random Forest 模型訓練與預測
from sklearn.ensemble import RandomForestClassifier

# 使用相同的數據分割方式
X_train_rf, X_val_rf, y_train_rf, y_val_rf = train_test_split(tfidf_matrix, y, test_size=0.1, random_state=42)

X_train_rf, X_val_rf, y_train_rf, y_val_rf = train_test_split(X_train_rf, y_train_rf, test_size=0.1, random_state=42)

# 創建隨機森林模型
rand_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 訓練隨機森林模型
rand_forest_model.fit(X_train_rf, y_train_rf)

# 預測
y_val_pred_rf = rand_forest_model.predict(X_val_rf)
y_test_pred_rf = rand_forest_model.predict(X_test)

# 分類報告
print("驗證集 Validation Classification Report:")
print(classification_report(y_val_rf, y_val_pred_rf))

print("\n測試集 Test Classification Report:")
print(classification_report(y_test, y_test_pred_rf))

# 對照表
result_df_val_rf = pd.DataFrame({'Actual': y_val_rf, 'Predicted': y_val_pred_rf})
result_df_test_rf = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred_rf})

print("驗證集 Validation Result Comparison:")
print(result_df_val_rf)

print("\n測試集 Test Result Comparison:")
print(result_df_test_rf)



In [ ]:
new_article = "IC 設計大廠聯發科 (2454-TW) 副董事長暨執行長蔡力行今 (26) 日獲頒潘文淵獎，會後受訪表示，聯發科 3 奈米會在台積電 (2330-TW)(TSM-US) 做，且由於先進製程技術相當複雜，不論要採用或更換都非常困難，雙方會持續緊密合作。外界今日提問不論是輝達 (NVDA-US)、蘋果 (AAPL-US) 等都表示尋求多元的晶圓代工方案，蔡力行回應，聯發科在先進製程持續與台積電緊密合作，英特爾 (INTC-US) 則負責 16 奈米蔡力行也強調，聯發科不會只停在採用 4 奈米，也會採用 3 奈米製程，此外，由於電晶體微縮速度趨緩，儘管技術上可行，但不一定符合經濟效益，因此技術也逐步從平面變成 2D、2.5D，甚至 3D 等，先進封裝的重要性比以前增加。至於跟輝達合作，蔡力行重申，雙方合作仍以汽車為主，輝達布局車用比聯發科早，主要著墨在智慧座艙與 ADAS 系統，雙方有很好的配合，其中，輝達主攻高階、聯發科則瞄準中階，雙方正密切合作開會。"

processed_new_article = tokenize_and_remove_stopwords(new_article)

print(processed_new_article)

# 將新文章轉換為 TF-IDF 表示形式
new_article_tfidf = tfidf_vectorizer.transform([processed_new_article])

# 使用投票分類器進行預測
predicted_label_ensemble = voting_classifier.predict(new_article_tfidf)

print(f"新文章預測結果: {predicted_label_ensemble}")


In [ ]:
# 使用 inverse_transform 將預測的數字編碼轉換回原始標籤
predicted_label_original = label_encoder.inverse_transform(predicted_label_ensemble)

print(f"新文章預測結果（原始標籤）: {predicted_label_original}")


## 實際預測（研究）


In [ ]:
# 獲取所有標籤對應的編碼
all_labels = label_encoder.classes_

print("所有標籤對應的編碼:")
for label_code, label in enumerate(all_labels):
    print(f"編碼 {label_code}: 標籤 {label}")


In [ ]:
import jieba

# 定義停用詞
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

# 定義分詞並去除停用詞的函數
def tokenize_and_remove_stopwords(text):
    words = jieba.cut(text)  # 使用 jieba 进行分词
    words_filtered = [word for word in words if word not in stopwords]  # 去除停用词
    return ' '.join(words_filtered)

# 將處理後的內容加入 DataFrame 中
df['processed_content'] = df['Content'].apply(tokenize_and_remove_stopwords)

# 新文章
new_article = "台股守穩季線，週線三連紅。（資料照） 〔財經頻道／綜合報導〕美國CPI略高於市場預期，美股漲勢暫歇，本週以來，台股經過兩日大漲後，今（13）日指數震盪走低，終場下跌43.34點，以16782.57點作收，守住季線關卡，成交量為2986.08億元，週線上漲262點，呈現三連紅，緯創失守百元大關，AI族群普遍都是收黑，電子類股以矽光子、網通等次族群比較有表現，傳產輪動到營建、造紙、百貨等接棒演出。 前10大成交額個股漲多跌少，除了AI族群收黑，其他都是紅盤居多，廣達跌12元，收226元，成交額182.32億元，排名第1；台積電終場漲3元，收553元，成交額171.29億元，排名第2；矽統終場漲2.75元，收47.7元，成交額146.25億元，排名第3；定穎投控漲3.3元，收103元，成交額95.92億元，排名第4；緯創跌3.4元，收99.1元，成交額93.89億元，排名第5。 請繼續往下閱讀...  技嘉跌13.5元，收271元，成交額89.05億元，排名第6；創意收1695元平盤，成交額86.78億元，排名第7；聯發科上漲27元，收842元，成交額81.63億元，排名第8；裕隆漲1.1元，收85.1元，成交額66.51億元，排名第9；材料-KY漲 5元，收1185元，成交額63.34億元，排名第10。 一手掌握經濟脈動點我訂閱自由財經Youtube頻道 不用抽 不用搶 現在用APP看新聞 保證天天中獎點我下載APP按我看活動辦法 相關新聞"

# 處理新文章
processed_new_article = tokenize_and_remove_stopwords(new_article)

# 輸出處理後的文章
print(processed_new_article)

# 將新文章轉換為 TF-IDF 表示形式
new_article_tfidf = tfidf_vectorizer.transform([processed_new_article])

# 使用投票分類器進行預測
predicted_label_ensemble = voting_classifier.predict(new_article_tfidf)

# 輸出預測結果
print(f"新文章預測結果: {predicted_label_ensemble}")


In [ ]:
import jieba

# 定義停用詞
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

# 定義分詞並去除停用詞的函數
def tokenize_and_remove_stopwords(text):
    words = jieba.cut(text)  # 使用 jieba 进行分词
    words_filtered = [word for word in words if word not in stopwords]  # 去除停用词
    return ' '.join(words_filtered)

# 將處理後的內容加入 DataFrame 中
df['processed_content'] = df['Content'].apply(tokenize_and_remove_stopwords)

# 新文章
new_article = "小漲"

# 處理新文章
processed_new_article = tokenize_and_remove_stopwords(new_article)

# 輸出處理後的文章
print(processed_new_article)

# 將新文章轉換為 TF-IDF 表示形式
new_article_tfidf = tfidf_vectorizer.transform([processed_new_article])

# 使用投票分類器進行預測
predicted_label_ensemble = voting_classifier.predict(new_article_tfidf)

# 輸出預測結果
print(f"新文章預測結果: {predicted_label_ensemble}")